In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import models.Transformer as model
from utils import DataConfig

np.random.seed(3407)
torch.manual_seed(3407)
torch.cuda.manual_seed_all(3407)
torch.backends.cudnn.deterministic = True  # 保证每次结果一样
torch.backends.cudnn.benchmark = False
UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号

In [3]:
data_config = DataConfig('100d', 'word2vec')
model_config = model.ModelConfig()

In [19]:
class Model(nn.Module):
    def __init__(self, model_config, data_config):
        super(Model, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(
            data_config.embedding_pretrained,
            freeze=False) if data_config.embedding_pretrained is not None else nn.Embedding(data_config.n_vocab,
                                                                                            data_config.embed,
                                                                                            padding_idx=data_config.n_vocab - 1)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=data_config.embed, nhead=model_config.num_head,
                                                        batch_first=True)
        self.encoders = nn.TransformerEncoder(self.encoder_layer, num_layers=model_config.num_encoder)
        self.fc = nn.Linear(data_config.embed, data_config.num_classes)

    def forward(self, x):
        out = self.embedding(x)
        out = self.encoders(out)
        # out = out.reshape(out.size(0), -1)
        out=out[:,0,:]
        out = self.fc(out)
        return out


In [20]:
model = Model(model_config, data_config).to(data_config.device)

In [21]:
model(torch.randint(1, 10, [2, 30]).to(data_config.device)).size()

torch.Size([2, 30, 100])
torch.Size([2, 100])


torch.Size([2, 5])

In [90]:
def init_network(model, method='xavier', exclude='embedding'):
    for name, w in model.named_parameters():
        if exclude not in name:  # 如果不是嵌入层
            if 'weight' in name:  # weight 三种初始化方式
                if method == 'xavier' and len(w.size() < 2):
                    nn.init.xavier_normal_(w)
                elif method == 'kaiming':
                    nn.init.kaiming_normal_(w)
                else:
                    nn.init.normal_(w)
            elif 'bias' in name:  # bias 置0
                nn.init.constant_(w, 0)
            else:
                pass

In [23]:
init_network(model)

In [14]:
from torchinfo import summary

summary(model, input_size=(1, 30), dtypes=[torch.long])

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 5]                    --
├─Embedding: 1-1                         [1, 30, 100]              72,700
├─Linear: 1-2                            [1, 5]                    15,005
Total params: 87,705
Trainable params: 87,705
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.09
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.35
Estimated Total Size (MB): 0.38

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取CSV文件
df = pd.read_csv('./ship_data/experiment_data.csv')  # 替换为你的CSV文件路径

# 划分数据集
train_df, temp_df = train_test_split(df, test_size=0.1, random_state=3407)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=3407)

# 保存划分后的数据集为新的CSV文件
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('val_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)


In [100]:
import torch
import torch.nn.functional as F

# 假设你的输入是一个大小为[2, 30, 100]的tensor
input_tensor = torch.randn(2, 30, 100)

# 执行最大池化操作，保留最大的两个值
output_tensor = F.max_pool1d(input_tensor, kernel_size=2, stride=1)

print(output_tensor.shape)  # 输出应为[2, 2, 100]


torch.Size([2, 30, 99])
